# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
July 30, 2020: Perform analysis with stored histograms.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

## Extract folder

In [5]:
img_size=128
dict1={'scratch':'/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{0}square/'.format(img_size),
    'proj':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
display(u)


interactive(children=(Select(description='x', options=('scratch', 'proj'), value='scratch'), Output()), _dom_c…

In [6]:
parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20200911_083711_bsize64_spec_test_128_nospec', '2020…

In [7]:
result=w.result
main_dir=parent_dir+result
print(main_dir)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210226_175209_bsize256_scale0.1_deterministic_off


In [8]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

In [9]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [10]:
# main_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/scaling_runs/20201031_205304_bsize512_scaling_2_8'

## Extract metrics info from log file

In [11]:
def f_extract_info(fname):
    '''
    Module to extract information from out.log files of Lbann training
    Reads in file name
    '''
    strg_lst=['objective','d_real','d_fake','gen','spec_loss','run time','mini-batch']
    keys=['training_'+strg for strg in strg_lst]
    dict1={}
    for category in ['training','validation']:
        for strg in strg_lst:
            try: 
                key=category+'_'+strg
                cmd='grep "{0}" {1} | grep "{2}"'.format(category,fname,strg)
        #         print(cmd)
                op1=sp.check_output(cmd,shell=True).decode('utf-8').split('\n')
                obj=np.array([strg.split(':')[-1] for strg in op1 if strg])
                dict1[key]=obj
            except Exception as e:
                print(e)
                dict1[key]=np.nan
    
    df=pd.DataFrame([])
    key_lst=['training_objective', 'training_d_real', 'training_d_fake', 'training_gen', 'training_spec_loss','validation_objective', 'validation_d_real', 'validation_d_fake', 'validation_gen','validation_spec_loss']
    col_list=['train_obj','train_dreal','train_dfake','train_gen','train_spec','val_obj','val_dreal','val_dfake','val_gen','val_spec']
    for col,key in zip(col_list,key_lst):
        try: 
            df[col]=dict1[key].astype(np.float)
        except: pass

    ### Need to remove the trailing 's' in the timings
    for col,key in zip(['train_time','val_time'],['training_run time','validation_run time']):
        df[col]=np.array([i[:-1] for i in dict1[key]]).astype(np.float)

    for col,key in zip(['train_batch_stats','val_batch_stats'],['training_mini-batch','validation_mini-batch']):
        df[col]=dict1[key]
        
    return df

def f_plot_metrics(df,col_list=['train_obj']):
    '''
    Plot multiple columns of the dataframe
    '''
    plt.figure()
    
    marker_lst=('o','*','H','D','.','x')
    marker=itertools.cycle(marker_lst)
    for col in col_list: plt.plot(df[col],linestyle='',marker=next(marker),label=col)
    plt.legend()
    plt.xlabel('Epoch')
    

In [12]:
strg=main_dir+'/out.log'
df_metrics=f_extract_info(glob.glob(strg)[0])

In [13]:
df_metrics.head()

,train_obj,train_dreal,train_dfake,train_gen,train_spec,val_obj,val_dreal,val_dfake,val_gen,val_spec,train_time,val_time,train_batch_stats,val_batch_stats
0,11.66370,1.121530,1.025180,8.20232,13.1471,6.07607,0.189825,0.261554,4.360130,12.6456,477.023,4.89749,"0.542463s mean, 13.2366s max, 0.0266994s min,...","0.0499295s mean, 0.674377s max, 0.0212096s mi..."
1,6.64313,0.488296,0.447395,4.50567,12.0177,15.37270,0.762649,0.065537,13.394000,11.5045,470.660,5.88349,"0.535233s mean, 12.4597s max, 0.028455s min, ...","0.0599905s mean, 0.749879s max, 0.016318s min..."
2,5.80714,0.370653,0.351939,3.92026,11.6428,3.11646,0.349514,0.175034,1.446930,11.4498,440.906,4.09537,"0.50138s mean, 9.7302s max, 0.0270251s min, 1...","0.0417448s mean, 0.434532s max, 0.0163489s mi..."
3,5.30511,0.340484,0.334236,3.47730,11.5309,11.15500,1.122940,0.080311,8.792970,11.5874,402.083,9.56473,"0.457215s mean, 11.3769s max, 0.026943s min, ...","0.0975533s mean, 3.51449s max, 0.0168001s min..."
4,4.95641,0.360317,0.353641,3.08922,11.5323,3.28328,0.135238,1.153760,0.841275,11.5301,422.864,4.94935,"0.480856s mean, 7.91464s max, 0.0272264s min,...","0.0504375s mean, 0.50381s max, 0.0159224s min..."


In [14]:
col_list=df_metrics.columns[~df_metrics.columns.str.endswith('stats')]
interact_manual(f_plot_metrics,col_list=SelectMultiple(options=col_list),df=fixed(df_metrics))


interactive(children=(SelectMultiple(description='col_list', options=('train_obj', 'train_dreal', 'train_dfake…

<function __main__.f_plot_metrics(df, col_list=['train_obj'])>

## Extract data from stored dataframe

In [15]:
### Load data
df=pd.read_pickle(main_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [16]:
print(df.shape)
df.tail()


(1758, 22)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec3,num_imgs,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_sdev,label
1753,19,17530,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.016795,0.001091,0.058260,0.076147,0.013404,6.087511,...,25.438133,256,4,0,"[1.015959556041075, 0.49171524383975546, 0.049...","[0.0033494240627420665, 0.001533796090566746, ...","[-1.031746031584782, -0.3099415213241682, 0.30...","[84338.63431543857, 58676.65387212038, 42728.1...","[43309.3794359084, 20898.61386535446, 14417.03...",19-17530
1754,19,17540,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.010847,0.002474,0.159623,0.172943,0.008171,7.094986,...,31.615143,256,9,0,"[1.0380432558012525, 0.47864579148425707, 0.05...","[0.003885703094345541, 0.0017754137606646328, ...","[-1.031746031584782, -0.3099415213241682, 0.30...","[111035.27460477129, 77283.4571272507, 51290.9...","[59874.85916587743, 29369.757913888134, 18153....",19-17540
1755,19,17550,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.029363,0.036083,0.022350,0.087796,0.002866,4.764689,...,30.012879,256,4,0,"[1.16100517214787, 0.4203802052417338, 0.03626...","[0.0035102543106102202, 0.0016756886190460673,...","[-1.031746031584782, -0.3099415213241682, 0.30...","[74767.48379244469, 53472.70199472668, 38472.6...","[35761.33483664293, 20320.52995603113, 12322.6...",19-17550
1756,19,17560,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.029918,0.042707,0.005229,0.077855,0.004295,3.946270,...,30.552298,256,1,0,"[1.0670748157010785, 0.4694204006256112, 0.041...","[0.0033437414877736274, 0.0015812673782240607,...","[-1.031746031584782, -0.3099415213241682, 0.30...","[71789.49536034092, 49444.251543575614, 33598....","[39651.10258336275, 17046.035196000055, 10301....",19-17560
1757,19,17570,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.001874,0.003002,0.043742,0.048618,0.000451,6.092245,...,27.866128,256,8,1,"[1.1003865338664094, 0.44727925981879935, 0.04...","[0.0031520897999214835, 0.001411843573512864, ...","[-1.031746031584782, -0.3099415213241682, 0.30...","[88485.26257414743, 59611.31071579913, 40434.2...","[53961.63840664043, 24471.44813873732, 13975.6...",19-17570


In [17]:
df.columns

Index(['epoch', 'step', 'img_type', 'fname', 'chi_1a', 'chi_1b', 'chi_1c',
       'chi_1', 'chi_2', 'chi_imgvar', 'chi_spec1', 'chi_spec2', 'chi_spec3',
       'num_imgs', 'num_large', 'num_vlarge', 'hist_val', 'hist_err',
       'hist_bin_centers', 'spec_val', 'spec_sdev', 'label'],
      dtype='object')

### View best epochs

#### Locations with best chi_sqr

In [18]:
def f_get_best_chisqr_models(df,cutoff=0.2):
    '''
    Pick models with lowest 20% chi-square for multiple categories
    '''
    chi_sqr_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[chi_sqr_keys])
#     print(q_dict)
    
    df_sliced=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    return df_sliced

chi_sqr_keys=['epoch','step','chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
# index location of min/max values of chi squares
inds=[]
for key in ['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']:
    inds.append(df[key].idxmin(axis=1))
inds=np.array(inds)
df.loc[inds][chi_sqr_keys]


,epoch,step,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2
681,7,6810,0.000389,0.001770,0.033989,0.036148,0.000116,6.664384,9.878868e-01,2.143855e+02
647,7,6470,0.013618,0.000711,0.069868,0.084198,0.008890,7.092261,1.258402e+00,7.140978e+01
624,7,6240,0.009065,0.020873,0.000582,0.030520,0.002751,5.006674,1.204620e+00,7.541957e+01
857,9,8570,0.003747,0.006825,0.000795,0.011366,0.001003,5.232429,5.408509e-01,5.474589e+01
793,9,7930,0.000791,0.003742,0.037148,0.041681,0.000073,6.385770,3.936523e-01,3.357695e+01
8,0,80,0.846258,0.108368,0.003864,0.958489,0.622217,0.497681,1.181336e+07,7.160662e+08
1396,15,13960,0.011831,0.023147,0.008171,0.043150,0.005629,4.914970,1.853094e-01,2.191053e+01
1471,16,14710,0.010240,0.009103,0.025903,0.045246,0.006257,5.989348,2.279620e-01,1.696681e+01


In [19]:
### Plot chi-sqr values
# _=df_sliced.plot(x="step", y=["chi_1", "chi_imgvar", "chi_spec1"],style='.',marker='*')

In [20]:
### Plot number of very high pixel images
plt.figure()
plt.plot(df[df.img_type=='train_gen'].step,df[df.img_type=='train_gen'].num_vlarge,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of very large pixel images from a batch of images')

# df[df.num_vlarge>10].tail()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Number of very large pixel images from a batch of images')

### View best steps

In [21]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [22]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [23]:
df_sliced=w.result
# df_sliced

### Pick best steps

In [24]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1b',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_spec1',head=4,display_flag=False).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.04282909502458844, 'chi_spec1': 0.6533325854721304}
[0, 19]
{'chi_1': 0.18868450002012596, 'chi_spec1': 3.5489015309356264}
[0, 19]
{'chi_1': 0.04282909502458844, 'chi_spec1': 0.6533325854721304}
[ 6470  7150  8570  9840 10290 10680 10930 11130 11980 12460]


In [25]:
# best_step=[3430,  6460,  7215,  9610, 11725, 14635, 9475]
# best_step=np.arange(40130,40135).astype(int)

In [26]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(10, 22)
[(7, 6470), (8, 7150), (9, 8570), (11, 9840), (11, 10290), (12, 10680), (12, 10930), (12, 11130), (13, 11980), (14, 12460)]


In [27]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
647,7-6470,0.084198,1.258402,30.758242,0.013618,0.000711,0.069868,0.008890,7.092261,71.409785,7,6470
715,8-7150,0.013449,0.345493,26.567803,0.001942,0.010415,0.001092,0.000136,5.648892,38.755703,8,7150
857,9-8570,0.011366,0.540851,28.503578,0.003747,0.006825,0.000795,0.001003,5.232429,54.745891,9,8570
984,11-9840,0.039169,0.314395,26.859539,0.004187,0.000712,0.034269,0.000627,6.318643,25.793158,11,9840
1029,11-10290,0.022715,0.264390,28.928519,0.003293,0.008333,0.011089,0.000967,5.700610,25.688772,11,10290
1068,12-10680,0.015498,0.593626,24.719442,0.003894,0.002134,0.009470,0.000262,6.311813,57.037761,12,10680
1093,12-10930,0.029626,0.274449,25.861979,0.002766,0.008717,0.018144,0.000673,5.263426,23.444355,12,10930
1113,12-11130,0.035578,0.277865,25.166647,0.005382,0.024704,0.005493,0.001057,4.600322,28.515385,12,11130
1198,13-11980,0.028194,0.243385,22.410408,0.005333,0.015432,0.007429,0.002041,5.053409,26.749091,13,11980
1246,14-12460,0.012438,0.558514,24.855876,0.001203,0.002869,0.008367,0.000225,5.737712,37.046050,14,12460


## Plot pixel intensity and spectrum

In [28]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=128
    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))

    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)
            
        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)
            plt.legend()
            
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=100)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')

    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
    
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [29]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('7-6470', '8-7150', '9-8570', '11-984…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [30]:
df[(df.step>8000)& (df.step<10000)].plot(kind='line',x='step',y=['chi_1','chi_spec1'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='step'>

## View image block

In [31]:
def f_get_img(df,step):
    df_temp=df[(df.step==step)]
    images=np.load(df_temp.fname.values[0])[:,0,:,:]
    return images

img_arr=f_get_img(df,8020)
# f_plot_grid(img_arr[:18],cols=6,fig_size=(10,5))

In [32]:
f_plot_grid(s_val[100:118],cols=6,fig_size=(10,5))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plotting spread of spectrum of input data

In [ ]:

# plt.figure()

# idxs=np.random.randint(0,s_val.shape[0],size=50)
# arr=s_val[idxs]
# Pk = f_batch_spectrum(arr)
# k=np.arange(Pk.shape[1])
# for count,i in enumerate(Pk):
#     plt.plot(i)
    
# Pk = f_batch_spectrum(img_arr)
# mean,std = np.mean(Pk, axis=0),np.std(Pk, axis=0)/np.sqrt(Pk.shape[0])
# plt.plot(k, mean, 'k:',label='generated image')


# plt.legend()

# plt.ylabel(r'$P(k)$')
# plt.xlabel(r'$k$')
# plt.title('Power Spectrum: 50 input images')
# plt.yscale('log')



In [ ]:
main_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/'
f1=main_dir+'20201208_165658_bsize256_scale0.1_deterministic_on/dump_outs/trainer0/model0/sgd.training.epoch.1.step.1000_gen_img_instance1_activation_output0.npy'
a1=np.load(f1)[:,0,:,:]
f2=main_dir+'20201209_055955_bsize256_scale0.1_deterministic_on/dump_outs/trainer0/model0/sgd.training.epoch.1.step.1000_gen_img_instance1_activation_output0.npy'
a2=np.load(f2)[:,0,:,:]

print(a1.shape)
print(a2.shape)


In [ ]:
# f_pixel_intensity(a1)
# f_pixel_intensity(a2)

f_compare_pixel_intensity([a1,a2],label_lst=['img1','img2'],bkgnd_arr=[],log_scale=True, normalize=True, mode='avg',bins=25, hist_range=None)

In [ ]:
df_best.columns

In [ ]:
f_invtransform(df_best.iloc[0].hist_bin_centers)
